In [5]:
#https://stackoverflow.com/a/54594071
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Display code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
''')


# Latin Epigraphy Scraper 2.0

*The Jupyter Notebook inteface allows you to query the EDCS and save the search result in a TSV file and plot the results on a map of the Roman Empire without any prior knowledge of programming.*

## Authors 
* [Brian Ballsun-Stanton, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0003-4932-7912)
* [Petra Heřmánková, Aarhus University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-6349-0540)
* [Ray Laurence, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-8229-1053)

## Description 

This programme extracts the output of a search query from the [Epigraphik-Datenbank  Clauss / Slaby (EDCS)](http://www.manfredclauss.de/) in a reproducible manner and saves it as a TSV file. The output can be also plotted the output to a map of the Roman Empire, along with the system of Roman Provinces, roads, and cities. 

## Instructions for use

@PETRA TODO :)

In [6]:
import warnings
warnings.filterwarnings('ignore')
from lat_epig.interface import makeScrapeInterface, Parseargs                    # loads the user friendly GUI
makeScrapeInterface()

# Floriano

Text(value='', description='Text 1:')

ToggleButtons(description='Operator', options=('and', 'or', 'not'), value='and')

Text(value='', description='Text 2:')

Text(value='', description='EDCS-ID:')

Text(value='', description='Publication:')

...

Button(description='Inscription Genus...', style=ButtonStyle())

SelectMultiple(description='Inscription genus / personal status:', layout=Layout(display='none'), options=(Non…

SelectMultiple(description='and not:', layout=Layout(display='none'), options=(None, 'Augusti/Augustae', 'carm…

Button(description='Get inscriptions!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

# Export map instructions

@Petra todo

In [7]:
# #https://discourse.jupyter.org/t/check-if-running-on-binder/635/6
# import socket

# hostname = socket.gethostname()
# #print(hostname)

# if "jupyter-mqancienthistor" not in hostname:
from lat_epig.map_interface import make_map_interface
make_map_interface()
# else:
#     display(HTML("<h1>PDF Map</h1>PDF Map currently disabled on binder because of memory issues. <br/>You should, if you want a PDF map, <a href='https://github.com/mqAncientHistory/EpigraphyScraperNotebook/blob/master/README.md#instructions-how-to-use-the-tool'>Install lat_epig locally in Docker</a> instead."))



Button(description='Update Data File List', layout=Layout(width='max-content'), style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='max-content'), options=(('2021-08-16-term1_tumulus-71.t…

Text(value='', description='Map Title:')

Dropdown(description='Basemap', index=7, layout=Layout(width='max-content'), options=(('60 BC provinces', 'rom…

RadioButtons(description='Show Provinces', layout=Layout(width='max-content'), options=(('All Provinces', True…

...

RadioButtons(description='Marker size for inscriptions', index=1, layout=Layout(width='max-content'), options=…

Checkbox(value=False, description='Greyscale for publication. (You should probably use a different basemap).',…

RadioButtons(description='Citation', index=1, layout=Layout(width='max-content'), options=(('I will cite this …

Button(description='Export Desired Map!', layout=Layout(width='max-content'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [ ]:
# Interactive map instructions

@Petra todo

In [8]:
from lat_epig.interactive_map_interface import make_i_map_interface
make_i_map_interface()

Button(description='Update Data File List', layout=Layout(width='max-content'), style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='max-content'), options=(('2021-08-16-term1_tumulus-71.t…

Button(description='Refresh Interactive Map!', layout=Layout(width='max-content'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [9]:
HTML('''
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Display code"></form>''')